# Import des packages

In [8]:
!pip install snowflake-connector-python==3.0.0
!pip install pyarrow==8.0.0
!pip install azure-keyvault-secrets
!pip install pandas
!pip install "snowflake-connector-python[pandas]"
!pip install pyod

  Using cached pyarrow-8.0.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (29.4 MB)
ERROR: azureml-dataset-runtime 1.44.0 has requirement pyarrow<6.0.1,>=0.17.0, but you'll have pyarrow 8.0.0 which is incompatible.
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
  Using cached pyarrow-10.0.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (36.0 MB)
ERROR: azureml-dataset-runtime 1.44.0 has requirement pyarrow<6.0.1,>=0.17.0, but you'll have pyarrow 10.0.1 which is incompatible.
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 8.0.0
    Uninstalling pyarrow-8.0.0:
      Successfully uninstalled pyarrow-8.0.0


In [9]:
#On importe les librairies qui vont nous servir pour le traitement de la base de données
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn #Module de data science / Machine Learning / Analyse prédictive
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline
import warnings
import snowflake.connector
import plotly.express as px

In [10]:
from azure.identity import ManagedIdentityCredential, DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from azureml.core import Workspace
from datetime import datetime,timedelta
from scipy import stats
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [11]:
ws = Workspace.from_config()
keyvault = ws.get_default_keyvault()
password = keyvault.get_secret(name="YMOLDU")

In [12]:
db= snowflake.connector.connect(
                user='ysekal',
                password=password,
                account='vw51723.west-europe.azure',
                warehouse='WH_LAB_STANDARD',
                timeout=20)

In [13]:
cur = db.cursor()

# Création de la table

In [14]:
#On appelle la table A_ITEM_SALES_DAILY
req = """ SELECT * from  CHRONODRIVE_PROD.DATA_MART.A_ITEM_SALES_DAILY WHERE to_date(ETL_date) = current_date()-1 """

In [15]:
cur.execute(req)
print(cur.sfqid)

01ab8d01-0201-e955-0000-9f4d04e145c2


In [16]:
df = cur.fetch_pandas_all()

## Création des tables par types de données 

In [17]:
#création de la table avec les données quantitatives 
df_quanti= df.select_dtypes(exclude= ["object","datetime64","bool"])
df_quanti.head()

,INVOICE_SHOP_ID,TOTAL_BILL_QUANTITY,TOTAL_ET_AMOUNT,TOTAL_ET_AMOUNT_DISCOUNT_EXCLUDED,TOTAL_ATI_AMOUNT,TOTAL_VAT_AMOUNT,TOTAL_DISCOUNT_ET_AMOUNT,TOTAL_DISCOUNT_ATI_AMOUNT,TOTAL_VIRTUAL_LOT_ET_AMOUNT,TOTAL_VIRTUAL_LOT_ATI_AMOUNT,TOTAL_LOYALTY_ATI_AMOUNT,TOTAL_VIRTUAL_LOT_LOYALTY_ATI_AMOUNT,ITEM_COST,DAILY_AVERAGE_SALE_30_DAYS,DAILY_AVERAGE_SALE_7_DAYS,TOTAL_MARGIN_AMOUNT,TOTAL_MARGIN_AMOUNT_DISCOUNT_EXCLUDED,COST_GOOD_SOLD
0,1040,1,5.26,5.26,5.55,0.29,0.0,0.0,0.0,0.0,0.0,0.0,3.62,0.60,0.43,1.64,1.64,3.62
1,1047,1,2.66,2.66,3.19,0.53,0.0,0.0,0.0,0.0,0.0,0.0,1.47,0.27,0.29,1.19,1.19,1.47
2,1007,1,2.70,2.70,2.85,0.15,0.0,0.0,0.0,0.0,0.0,0.0,1.88,1.40,1.57,0.82,0.82,1.88
3,1048,1,1.00,1.00,1.05,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.82,0.97,0.43,0.18,0.18,0.82
4,1049,1,2.32,2.32,2.45,0.13,0.0,0.0,0.0,0.0,0.0,0.0,1.82,1.10,1.14,0.50,0.50,1.82


In [18]:
#création de la table avec les données qualitatives 
df_quali= df.select_dtypes(exclude= ["float64", "int16","int64","datetime64"])
df_quali.head()

,ITEM_ID,INVOICE_DATE,BBD_FLAG,SALE_TYPE,ETL_SOURCE,ETL_FLOW
0,52336,2023-04-09,False,MARC,"DATA_HUB & F_ORDER_HEADER, F_ORDER_DETAIL, F_I...",MJ_A_DATAHUB_DATAMART_ITEM_SALES_DAILY(2.4)
1,473526,2023-04-09,False,MARC,"DATA_HUB & F_ORDER_HEADER, F_ORDER_DETAIL, F_I...",MJ_A_DATAHUB_DATAMART_ITEM_SALES_DAILY(2.4)
2,171434,2023-04-09,False,MARC,"DATA_HUB & F_ORDER_HEADER, F_ORDER_DETAIL, F_I...",MJ_A_DATAHUB_DATAMART_ITEM_SALES_DAILY(2.4)
3,194224,2023-04-09,False,MARC,"DATA_HUB & F_ORDER_HEADER, F_ORDER_DETAIL, F_I...",MJ_A_DATAHUB_DATAMART_ITEM_SALES_DAILY(2.4)
4,187291,2023-04-09,False,MARC,"DATA_HUB & F_ORDER_HEADER, F_ORDER_DETAIL, F_I...",MJ_A_DATAHUB_DATAMART_ITEM_SALES_DAILY(2.4)


In [19]:
# Encoder les données quali
#La fonction get_dummies permet de créer des dummies à partir d'une variable catégorielle: On aura alors un DataFrame avec toutes les catégories encodées de façon binaire
warnings.filterwarnings('ignore')
df_quali['BBD_FLAG']=pd.get_dummies(df_quali['BBD_FLAG'])

In [20]:
warnings.filterwarnings('ignore')
df_quali['SALE_TYPE']=pd.get_dummies(df_quali['SALE_TYPE'])

## Aperçu des valeurs statistiques des tables 

In [21]:
df.describe()

,INVOICE_SHOP_ID,TOTAL_BILL_QUANTITY,TOTAL_ET_AMOUNT,TOTAL_ET_AMOUNT_DISCOUNT_EXCLUDED,TOTAL_ATI_AMOUNT,TOTAL_VAT_AMOUNT,TOTAL_DISCOUNT_ET_AMOUNT,TOTAL_DISCOUNT_ATI_AMOUNT,TOTAL_VIRTUAL_LOT_ET_AMOUNT,TOTAL_VIRTUAL_LOT_ATI_AMOUNT,TOTAL_LOYALTY_ATI_AMOUNT,TOTAL_VIRTUAL_LOT_LOYALTY_ATI_AMOUNT,ITEM_COST,DAILY_AVERAGE_SALE_30_DAYS,DAILY_AVERAGE_SALE_7_DAYS,TOTAL_MARGIN_AMOUNT,TOTAL_MARGIN_AMOUNT_DISCOUNT_EXCLUDED,COST_GOOD_SOLD
count,419690.000000,419690.000000,419690.000000,419690.000000,419690.000000,419690.000000,419690.000000,419690.000000,419690.000000,419690.000000,419690.000000,419690.000000,419234.000000,419690.000000,419690.000000,419690.000000,419690.000000,419690.000000
mean,1032.309326,2.607239,7.272318,7.600857,7.876432,0.624148,0.143196,0.157736,0.203238,0.223271,0.021852,0.025294,2.534244,2.293046,2.370136,1.355876,1.683498,5.908002
std,20.745012,3.966275,10.479182,11.253910,11.434270,1.265262,1.207121,1.375507,1.493601,1.653736,0.391092,0.402227,4.082558,3.947620,4.077977,4.417616,4.325992,10.161382
min,1001.000000,0.000000,-0.050000,0.000000,-0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2014.680000,-2014.680000,0.000000
25%,1016.000000,1.000000,2.560000,2.610000,2.790000,0.160000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.340000,0.940000,0.860000,0.380000,0.480000,1.890000
50%,1032.000000,2.000000,4.310000,4.450000,4.690000,0.300000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.910000,1.030000,1.140000,0.870000,0.990000,3.300000
75%,1047.000000,3.000000,8.060000,8.400000,8.750000,0.640000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.840000,2.700000,2.710000,1.740000,1.960000,6.420000
max,1077.000000,318.000000,1416.600000,1696.600000,1700.000000,283.400000,350.000000,420.000000,127.500000,153.090000,51.290000,57.600000,2023.050000,192.000000,206.860000,185.830000,185.830000,2023.050000


In [22]:
df_quanti.describe()

,INVOICE_SHOP_ID,TOTAL_BILL_QUANTITY,TOTAL_ET_AMOUNT,TOTAL_ET_AMOUNT_DISCOUNT_EXCLUDED,TOTAL_ATI_AMOUNT,TOTAL_VAT_AMOUNT,TOTAL_DISCOUNT_ET_AMOUNT,TOTAL_DISCOUNT_ATI_AMOUNT,TOTAL_VIRTUAL_LOT_ET_AMOUNT,TOTAL_VIRTUAL_LOT_ATI_AMOUNT,TOTAL_LOYALTY_ATI_AMOUNT,TOTAL_VIRTUAL_LOT_LOYALTY_ATI_AMOUNT,ITEM_COST,DAILY_AVERAGE_SALE_30_DAYS,DAILY_AVERAGE_SALE_7_DAYS,TOTAL_MARGIN_AMOUNT,TOTAL_MARGIN_AMOUNT_DISCOUNT_EXCLUDED,COST_GOOD_SOLD
count,419690.000000,419690.000000,419690.000000,419690.000000,419690.000000,419690.000000,419690.000000,419690.000000,419690.000000,419690.000000,419690.000000,419690.000000,419234.000000,419690.000000,419690.000000,419690.000000,419690.000000,419690.000000
mean,1032.309326,2.607239,7.272318,7.600857,7.876432,0.624148,0.143196,0.157736,0.203238,0.223271,0.021852,0.025294,2.534244,2.293046,2.370136,1.355876,1.683498,5.908002
std,20.745012,3.966275,10.479182,11.253910,11.434270,1.265262,1.207121,1.375507,1.493601,1.653736,0.391092,0.402227,4.082558,3.947620,4.077977,4.417616,4.325992,10.161382
min,1001.000000,0.000000,-0.050000,0.000000,-0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2014.680000,-2014.680000,0.000000
25%,1016.000000,1.000000,2.560000,2.610000,2.790000,0.160000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.340000,0.940000,0.860000,0.380000,0.480000,1.890000
50%,1032.000000,2.000000,4.310000,4.450000,4.690000,0.300000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.910000,1.030000,1.140000,0.870000,0.990000,3.300000
75%,1047.000000,3.000000,8.060000,8.400000,8.750000,0.640000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.840000,2.700000,2.710000,1.740000,1.960000,6.420000
max,1077.000000,318.000000,1416.600000,1696.600000,1700.000000,283.400000,350.000000,420.000000,127.500000,153.090000,51.290000,57.600000,2023.050000,192.000000,206.860000,185.830000,185.830000,2023.050000


In [23]:
df_quali.describe()

,BBD_FLAG,SALE_TYPE
count,419690.000000,419690.000000
mean,0.981744,0.000105
std,0.133877,0.010239
min,0.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,0.000000
75%,1.000000,0.000000
max,1.000000,1.000000


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419690 entries, 0 to 419689
Data columns (total 25 columns):
 #   Column                                 Non-Null Count   Dtype         
---  ------                                 --------------   -----         
 0   INVOICE_SHOP_ID                        419690 non-null  int16         
 1   ITEM_ID                                419690 non-null  object        
 2   INVOICE_DATE                           419690 non-null  object        
 3   TOTAL_BILL_QUANTITY                    419690 non-null  int16         
 4   TOTAL_ET_AMOUNT                        419690 non-null  float64       
 5   TOTAL_ET_AMOUNT_DISCOUNT_EXCLUDED      419690 non-null  float64       
 6   TOTAL_ATI_AMOUNT                       419690 non-null  float64       
 7   TOTAL_VAT_AMOUNT                       419690 non-null  float64       
 8   TOTAL_DISCOUNT_ET_AMOUNT               419690 non-null  float64       
 9   TOTAL_DISCOUNT_ATI_AMOUNT              419690 no

## Gestion des valeurs manquantes 

In [25]:
# Faire apparaitre sous forme de tableau les valeurs manquantes
dfVM=df_quanti.isnull().sum()
dfVM = dfVM.drop(dfVM[dfVM == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Valeurs manquantes' : dfVM})
missing_data

,Valeurs manquantes
ITEM_COST,456


In [26]:
#Remplacement des valeurs null par les valeurs médian.
df_quanti['ITEM_COST'] = df_quanti['ITEM_COST'].fillna(df_quanti['ITEM_COST'].median())

In [27]:
# Faire apparaitre sous forme de tableau les valeurs manquantes
dfVM=df_quanti.isnull().sum()
dfVM = dfVM.drop(dfVM[dfVM == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Valeurs manquantes' : dfVM})
missing_data

,Valeurs manquantes


# Valeurs négatives

On veut mettre dans cette liste toutes les colonnes où les données ne doivent pas être négatives.  
**Point d'attention** : il y a des colonnes qui auront des valeurs négatives comme valeurs 'normales'

In [28]:
liste_col_negative = []
for cols in df_quanti:
    if ((df_quanti[cols]<0).sum())>0:
        liste_col_negative.append(cols)
print(liste_col_negative)       

['TOTAL_ET_AMOUNT', 'TOTAL_ATI_AMOUNT', 'TOTAL_MARGIN_AMOUNT', 'TOTAL_MARGIN_AMOUNT_DISCOUNT_EXCLUDED']


# Outliers données qualitatives

## Gestion de la qualité du format date

In [29]:
list_errors = []
list_date = df_quali["INVOICE_DATE"].tolist()
for date in list_date:
    try:
        datetime.strftime(date, '%Y-%m-%d')
    except ValueError:
        list_errors.append(date)

In [30]:
df_quali['validate_date'] = np.where(df_quali['INVOICE_DATE'] in list_errors, False, True)
df_quali.head()
#df_quali[df_quali['validate_date']==False] 

,ITEM_ID,INVOICE_DATE,BBD_FLAG,SALE_TYPE,ETL_SOURCE,ETL_FLOW,validate_date
0,52336,2023-04-09,1,0,"DATA_HUB & F_ORDER_HEADER, F_ORDER_DETAIL, F_I...",MJ_A_DATAHUB_DATAMART_ITEM_SALES_DAILY(2.4),True
1,473526,2023-04-09,1,0,"DATA_HUB & F_ORDER_HEADER, F_ORDER_DETAIL, F_I...",MJ_A_DATAHUB_DATAMART_ITEM_SALES_DAILY(2.4),True
2,171434,2023-04-09,1,0,"DATA_HUB & F_ORDER_HEADER, F_ORDER_DETAIL, F_I...",MJ_A_DATAHUB_DATAMART_ITEM_SALES_DAILY(2.4),True
3,194224,2023-04-09,1,0,"DATA_HUB & F_ORDER_HEADER, F_ORDER_DETAIL, F_I...",MJ_A_DATAHUB_DATAMART_ITEM_SALES_DAILY(2.4),True
4,187291,2023-04-09,1,0,"DATA_HUB & F_ORDER_HEADER, F_ORDER_DETAIL, F_I...",MJ_A_DATAHUB_DATAMART_ITEM_SALES_DAILY(2.4),True


In [31]:
df_quali.groupby('validate_date').count()

,ITEM_ID,INVOICE_DATE,BBD_FLAG,SALE_TYPE,ETL_SOURCE,ETL_FLOW
validate_date,,,,,,
True,419690,419690,419690,419690,419690,419690


## Gestion de la qualité du format ID

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419690 entries, 0 to 419689
Data columns (total 25 columns):
 #   Column                                 Non-Null Count   Dtype         
---  ------                                 --------------   -----         
 0   INVOICE_SHOP_ID                        419690 non-null  int16         
 1   ITEM_ID                                419690 non-null  object        
 2   INVOICE_DATE                           419690 non-null  object        
 3   TOTAL_BILL_QUANTITY                    419690 non-null  int16         
 4   TOTAL_ET_AMOUNT                        419690 non-null  float64       
 5   TOTAL_ET_AMOUNT_DISCOUNT_EXCLUDED      419690 non-null  float64       
 6   TOTAL_ATI_AMOUNT                       419690 non-null  float64       
 7   TOTAL_VAT_AMOUNT                       419690 non-null  float64       
 8   TOTAL_DISCOUNT_ET_AMOUNT               419690 non-null  float64       
 9   TOTAL_DISCOUNT_ATI_AMOUNT              419690 no

In [33]:
#Liste des colonnes 
df_list= list(df.columns.values)

In [34]:
df_list = pd.DataFrame(df_list)

In [35]:
df_list=df_list.rename(columns=
               {0:'colname'})

In [36]:
df_id = df_list[df_list['colname'].str.contains('ID')]

In [37]:
list_id = list(df_id.colname.values)

In [38]:
for i in list_id :
    for columns in df.columns :
        if columns == i :
            df[columns] = df[columns].astype(str)

In [39]:
df.dtypes

INVOICE_SHOP_ID                                  object
ITEM_ID                                          object
INVOICE_DATE                                     object
TOTAL_BILL_QUANTITY                               int16
TOTAL_ET_AMOUNT                                 float64
TOTAL_ET_AMOUNT_DISCOUNT_EXCLUDED               float64
TOTAL_ATI_AMOUNT                                float64
TOTAL_VAT_AMOUNT                                float64
TOTAL_DISCOUNT_ET_AMOUNT                        float64
TOTAL_DISCOUNT_ATI_AMOUNT                       float64
TOTAL_VIRTUAL_LOT_ET_AMOUNT                     float64
TOTAL_VIRTUAL_LOT_ATI_AMOUNT                    float64
TOTAL_LOYALTY_ATI_AMOUNT                        float64
TOTAL_VIRTUAL_LOT_LOYALTY_ATI_AMOUNT            float64
ITEM_COST                                       float64
BBD_FLAG                                           bool
SALE_TYPE                                        object
DAILY_AVERAGE_SALE_30_DAYS                      

In [40]:
list_errors = []
for i in list_id:
    try:
        df[i].str.contains('\d{4,6}')
    except ValueError:
        list_errors.append(i)
df['validate_id'] = np.where(df[i] in list_errors, False, True)

In [41]:
df_quali.groupby('validate_id').count()

KeyError: 'validate_id'

# Outliers données quantitatives

# Introduction

Pourquoi et comment rechercher les valeurs aberrantes
De nombreux algorithmes d'apprentissage automatique et modèles de régression sont sensibles aux valeurs aberrantes. Une valeur aberrante est un point de données qui s'écarte considérablement des autres points. Si elles ne sont pas correctement prises en compte, les déductions obtenues à partir des modèles statistiques utilisés risquent de ne pas être utiles.

Il existe de nombreuses méthodes populaires pour détecter les valeurs aberrantes, à savoir les méthodes du score Z et de l'intervalle interquartile. Ces méthodes sont efficaces lorsque les données sous-jacentes suivent une distribution normale (une distribution dans laquelle la plupart des points de données sont plus proches de la moyenne et deviennent moins fréquents à mesure que la distance par rapport à la moyenne augmente). Toutefois, si les données ne sont pas distribuées normalement, ces méthodes peuvent classer à tort les observations normales comme des valeurs aberrantes. 

En revanche, la méthode Isolation Forest est non paramétrique, ce qui signifie simplement que nous n'avons pas à faire d'hypothèses sur la façon dont les données sous-jacentes sont distribuées.

En outre, les méthodes du score Z et de l'intervalle interquartile identifient au niveau de la variable. Si vous avez des raisons de croire que plusieurs variables interagissent entre elles et créent des valeurs aberrantes, ces méthodes ne seront pas en mesure de les détecter. 

Par exemple, un score SAT de 1350/1600 (90e percentile) ne semble pas être une valeur aberrante en soi. Cependant, si nous introduisons une autre dimension, l'âge, et que nous constatons qu'un enfant de 12 ans a obtenu 1350/1600, cette observation est probablement une valeur aberrante pour un sous-échantillon d'enfants de 12 ans. Contrairement aux méthodes de détection des valeurs aberrantes à une seule variable, Isolation Forest détecte les valeurs aberrantes dans un espace multidimensionnel.

# IQR

##  Inter-Quartile Range Method par ligne

La méthode de l'intervalle interquartile, illustrée au mieux par un diagramme en boîte, divise les données en quartiles en définissant trois points :

Le quartile 1 (Q1) représente le 25e percentile Le quartile 2 (Q2) représente le 50e percentile Le quartile 3 (Q3) représente le 75e percentile

La boîte dans le diagramme en boîte représente l'intervalle IQR qui est défini comme l'intervalle entre Q1 et Q3 ; IQR = Q3 - Q1 et les points de données qui tombent en dessous de Q1 - 1,5*IQR ou au-dessus de Q3 + 1,5*IQR sont définis comme des valeurs aberrantes.

Dans le diagramme en boîte, Q1 - 1,5*IQR et Q3 + 1,5*IQR sont représentés par les moustaches et les valeurs aberrantes sont représentées par des points au-dessus ou au-dessous.

In [ ]:
for col in df_quanti:
    df_quanti[col]= df_quanti[col].replace([0],np.NaN)

In [ ]:
q1 = df_quanti[col].quantile(0.25)
q3 = df_quanti[col].quantile(0.75)

IQR = q3 - q1

low_lim = q1 - 1.5 * IQR
up_lim = q3 + 1.5 * IQR

outlier =[]

df_quanti['outlier_IQR']=1
for col in df_quanti:
    if ((df_quanti[col].max()> up_lim) or (df_quanti[col].min()<low_lim)):
        df_quanti['outlier_IQR']=-1
        outlier.append(col)
print(' outlier in the dataset is', outlier)

In [ ]:
# test  IQR par ligne 
df_quanti['outlier_IQR']=1

for col in df_quanti:
    df.loc[df_quanti[col] > up_lim, "outlier_IQR"] = -1

In [ ]:
df_quanti[df_quanti['outlier_IQR']==-1].shape[0]

In [ ]:
df_quanti.groupby('outlier_IQR').count()

## Scoring IQR par colonne 

In [ ]:
list_col_outliers =[]
count_outliers = []
for col in df_quanti:
    columns = 0
    if ((df_quanti[col].max()> up_lim) or (df_quanti[col].min()<low_lim)):
        count_outliers.append(columns + 1)
        list_col_outliers.append(col)
print(list_col_outliers)
print(sum(count_outliers))
    

In [ ]:
import warnings
warnings.filterwarnings('ignore')
df_outlier = df
liste_col_outlier = []
for col in df_quanti:
    q1 = df_quanti[col].quantile(0.25)
    q3 = df_quanti[col].quantile(0.75)
    IQR = q3 - q1
    low_lim = q1 - 1.5 * IQR
    up_lim = q3 + 1.5 * IQR
    
    conditionlist = [
        (df_outlier[col]>= up_lim) | (df_outlier[col]<=low_lim),
        (df_outlier[col]< up_lim) & (df_outlier[col]>low_lim)]
    choicelist = [-1,0]
    df_outlier[col+'_IQR_score'] = np.select(conditionlist, choicelist, default= 0)
    if ((df_outlier[col].max()> up_lim) or (df_outlier[col].min()<low_lim)):
            liste_col_outlier.append(col)
df_outlier = df_outlier.iloc[:,-len(liste_col_outlier):]
(df_outlier.sum())*-1

In [ ]:
warnings.filterwarnings('ignore')
df['outlier_IQR'] = 0
for col in df_quanti:
    q1 = df_quanti[col].quantile(0.25)
    q3 = df_quanti[col].quantile(0.75)
    IQR = q3 - q1
    low_lim = q1 - 1.5 * IQR
    up_lim = q3 + 1.5 * IQR
    conditionlist = [
        (df_quanti[col]> up_lim) | (df_quanti[col]<low_lim),
        (df_quanti[col]<= up_lim) & (df_quanti[col]>=low_lim) & (df_quanti['outlier_IQR']!=1) ]
    choicelist = [1,0]
    df['outlier_IQR'] = np.select(conditionlist, choicelist, default= 0) 

In [ ]:
warnings.filterwarnings('ignore')
# Outlier par colonne
df_outlier = df_quanti
#Remplacement des valeurs null par les valeurs médian.
for col in df_quanti:
    df_outlier[col] =df_outlier[col].fillna(df_outlier[col].median())
liste_col_outlier = []
limit = []
for col in df_quanti:
    q1 = df_quanti[col].quantile(0.25)
    q3 = df_quanti[col].quantile(0.75)
    IQR = q3 - q1
    low_lim = q1 - 1.5 * IQR
    up_lim = q3 + 1.5 * IQR
    limit.append(up_lim)
    conditionlist = [
        (df_outlier[col]> up_lim) | (df_outlier[col]<low_lim),
        (df_outlier[col]<= up_lim) & (df_outlier[col]>=low_lim)]
    choicelist = [1,0]
    df_outlier[col+'_IQR_score'] = np.select(conditionlist, choicelist, default= 0)
    if ((df_outlier[col].max()> up_lim) or (df_outlier[col].min()<low_lim)):
            liste_col_outlier.append(col)
df_outlier = df_outlier.iloc[:,-len(liste_col_outlier):]
result = round(((df_outlier.sum())/df_outlier.shape[0])*100,2)
result = pd.DataFrame(data=result).reset_index()
result.rename(columns={"index": "Column name", 0: "% of outlier"})

# Z-Score 


La première étape est de détecter les valeurs aberrantes à l'aide de la méthode du score Z.

La méthode du score Z est efficace pour traiter les valeurs aberrantes pour les points de données qui suivent une distribution normale.

Le score Z indique la distance d'un point de données par rapport à la moyenne en tant que nombre d'écarts types.

On suppose que les observations dont le score Z est inférieur à -2,5 ou supérieur à 2,5 (c'est-à-dire 2,5 écarts types par rapport à la moyenne ; 1% de l'échantillon) sont des valeurs aberrantes.

In [ ]:
# (Number of entries, Number of features) Il y a 4626939 entrées et 18 variable
print(df_quanti.shape)

In [ ]:
for col in df_quanti:
    df_quanti['outlier_zscore']=0
    upper_limit = df_quanti[col].mean() + 3 *  df[col].std()
    lower_limit = df_quanti[col].mean() - 3 *  df[col].std()


    conditionlist = [
        (df_quanti[col] >= upper_limit),
        (df_quanti[col] <= lower_limit),
        (df_quanti[col] < upper_limit) & (df_quanti[col] > lower_limit) & (df_quanti['outlier_zscore']!=-1)
    ]
    choicelist = [-1, -1,0]

    df['outlier_zscore'] = np.select(conditionlist, choicelist, default= 0)

In [ ]:
df_quanti.groupby(['outlier_zscore']).count()

Si l'on revient aux histogrammes ci-dessus, on constate que les historammes sont  significativement orientés vers la gauche. Dans ce cas, la méthode du score Z ou d'autres méthodes populaires de détection des valeurs aberrantes, telles que la méthode de l'intervalle interquartile (IQR), n'est d'aucune utilité. Pour résoudre ce problème, j'effectue des transformations logarithmiques sur ces variables pour voir si je peux les décrire avec une distribution normale.

L'histogramme montre également que les quatre variables comportent de nombreux zéros (ou de très petites valeurs), ce qui est logique d'un point de vue économique. Par conséquent, je ne chercherai que les valeurs aberrantes du côté droit de la distribution.

Tout d'abord, je remplacerai les zéros par des NaN. Cela ne pose pas de problème car les zéros ne seront pas considérés comme des valeurs aberrantes.

# Isolation Forest - Multidimensional dimension 

Isolation Forest est un **algorithme d'apprentissage non supervisé** qui appartient à la famille des arbres de décision pour **détecter les anomalies**, proposée pour la première fois par Liu, Ting et Zhou (2008). 

Contrairement à d'autres méthodes qui tentent d'abord de **comprendre les points normaux** puis identifiaient tout ce qui se trouvait **en dehors de cette région** définie comme une **valeur aberrante ou anormale**.

Cette méthode fonctionne différemment. Elle **isole explicitement les anomalies** au lieu d'établir des profils et de construire des points et des régions normaux en attribuant un score à chaque point de données. **Cet algorithme** fonctionne parfaitement avec des **ensembles de données de très grande dimension** et s'est avéré être un moyen très efficace de **détecter les anomalies**. 

Les anomalies présentent deux caractéristiques. Elles sont éloignées des points normaux et sont peu nombreuses. L'algorithme Isolation Forest exploite ces deux caractéristiques.

**Avantages :**

- L'Isolation Forest ne suppose pas une distribution normale et est capable de détecter les valeurs aberrantes à un niveau multidimensionnel.
- Isolation Forest est efficace en termes de calcul : Par conséquent, il s'adapte bien aux grands ensembles de données.
- La forêt d'isolement est plus performante que la forêt aléatoire, en particulier pour les grands ensembles de données.

**Inconvénient :**

- L'algorithme Isolation Forest exige que nous choisissions le pourcentage d'anomalies dans l'ensemble de données. Nous devons donc avoir au moins une idée de la proportion d'anomalies dans nos données.
- Dans le pca on est obligé de réduire les dimensions à 2 pour bien performer, avec l'isolation forest on n'est pas obligé de faire ça car ça marche très bien avec plusieurs dimensions

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MaxAbsScaler

In [ ]:
#Remplacement des valeurs null par les valeurs médian.
df_quanti['ITEM_COST'] =df_quanti['ITEM_COST'].fillna(df_quanti['ITEM_COST'].median())

In [ ]:
sample_isf = df_quanti.drop(columns='INVOICE_SHOP_ID')

In [ ]:
# Define dimensions = 2
pca = PCA(n_components=2)

In [ ]:
# Conduct the PCA
principal_comp = pca.fit_transform(sample_isf)

On va mettre nos résulatts en data frame afin de pouvoir obtenir un score par la suite 

In [ ]:
# Convert to dataframe
pca_df = pd.DataFrame(data = principal_comp, columns = ['principal_component_1', 'principal_component_2'])
pca_df.head()

Entrainer le modèle et faire nos prédictions, on utilise un pourcentage de 4% (je ne sais pas trop pk pour l'instant

In [ ]:
# Train the model
isf = IsolationForest(contamination=0.02)
isf.fit(pca_df)

In [ ]:
# Predictions
predictions = isf.predict(pca_df)

On va ensuite mettre en place des scores grace à nos prédictions 

In [ ]:
# Extract scores
pca_df["iso_forest_scores"] = isf.decision_function(pca_df)

In [ ]:
# Extract predictions
pca_df["iso_forest_outliers"] = predictions

In [ ]:
pca_df.describe()

## Graphique 

In [ ]:
# Create a function to plot firms on the 2-dimensional space
def plot_firms (dataframe, title, color = None):
    """ Plot firms on the 2-dimensional space """
    
    # Generate a scatter plot
    fig = px.scatter(pca_df, x="principal_component_1", y="principal_component_2", title=title, color=color)
    
    # Layout
    fig.update_layout(
        font_family='Arial Black',
        title=dict(font=dict(size=20, color='red')),
        yaxis=dict(tickfont=dict(size=13, color='black'),
                   titlefont=dict(size=15, color='black')),
        xaxis=dict(tickfont=dict(size=13, color='black'),
                   titlefont=dict(size=15, color='black')),
        legend=dict(font=dict(size=10, color='black')),
        plot_bgcolor='white'
    )
      
    return(fig)

In [ ]:
# Need to import renderers to view the plots on GitHub
import plotly.io as pio

In [ ]:
# Plot [1] All firms
plot_firms(pca_df, "Figure 1: All Firms").show('png')

In [ ]:
# [2] Normal Firms vs. Outlier Firms
plot_firms(dataframe=pca_df, title="Figure 2: Normal Firms vs. Outlier Firms", color='iso_forest_outliers').show('png')

In [ ]:
# [3] Isolation Forest Scores
plot_firms(dataframe=pca_df, title="Figure 3: Isolation Forest Scores", color='iso_forest_scores').show("png")

In [ ]:
# Add identifiers and cluster assignments (labels) to the sample
pca_df = pd.concat([df_quanti['INVOICE_SHOP_ID'], pca_df], axis=1)

In [ ]:
# Print the first 5 firms
pca_df.head()

In [ ]:
df_quanti["iso_forest_outliers"] = pca_df["iso_forest_outliers"]

In [ ]:
df_quanti.groupby('iso_forest_outliers').count()

# LOF

Reminder sur les paramètres du LOF : https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.LocalOutlierFactor.html

 * n_neighbors int, default=20
Number of neighbors to use by default for kneighbors queries. If n_neighbors is larger than the number of samples provided, all samples will be used.

* metric str or callable, default=’minkowski’
Metric to use for distance computation. Default is “minkowski”, which results in the standard Euclidean distance when p = 2. See the documentation of scipy.spatial.distance and the metrics listed in distance_metrics for valid metric values.

* contamination ‘auto’ or float, default=’auto’
The amount of contamination of the data set, i.e. the proportion of outliers in the data set. When fitting this is used to define the threshold on the scores of the samples.
if ‘auto’, the threshold is determined as in the original paper,
if a float, the contamination should be in the range (0, 0.5].

In [ ]:
from pyod.models.iforest import IForest
from pyod.utils.data import generate_data
from pyod.utils.data import evaluate_print
from pyod.utils.example import visualize
from sklearn.neighbors import LocalOutlierFactor

In [ ]:
# model specification
model1 = LocalOutlierFactor(n_neighbors = 35 , metric = "minkowski", contamination = 0.02)
# model fitting
y_pred = model1.fit_predict(df_quanti)

In [ ]:
print("Nombre d'outliers détectés : ", len(y_pred[y_pred==-1]), ' sur un échantillon de ', df_quanti.shape[0], ' lignes')

In [ ]:
df_quanti["lof_outliers"] = y_pred 

In [ ]:
colors = df_quanti['lof_outliers']
plt.scatter(df_quanti["TOTAL_ET_AMOUNT"], df_quanti["TOTAL_ET_AMOUNT_DISCOUNT_EXCLUDED"], c = colors)

# produce a legend with the unique colors from the scatter
legend = plt.legend([-1,0], title = "Legend")